In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark import StorageLevel
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis import Extent
from geopyspark.geotrellis.constants import SPATIAL, NODATAINT, MAX, SQUARE, EXACT, ZOOM
from geopyspark.geotrellis.rdd import TiledRasterRDD
from geopyspark.geotrellis.pyramid import Pyramid
from geopyspark.geotrellis.constants import *
import json
import shapely
from shapely.geometry import shape, asShape, MultiPoint, MultiLineString
import matplotlib.pyplot as plt

from geonotebook.wrappers import RddRasterData

%matplotlib inline

conf = SparkConf()
conf.setMaster("local[*]")

gps = GeoPyContext(SparkContext.getOrCreate(conf))

In [ ]:
with open('roads.geojson', encoding='utf-8') as data_file:
    roads = json.load(data_file)
    libya_roads = MultiLineString([shape(geom['geometry']) for geom in roads['features']])

with open('Libya_shape.geojson', encoding='utf-8') as data_file:
    libya_border = shape(json.load(data_file)['features'][0]['geometry'])

In [ ]:
libya_border.bounds
from geopyspark import geotrellis
Extent(*libya_border.bounds)
Extent(*libya_roads.bounds)

In [ ]:
# Why does this call take any time at all?
from geopyspark.geotrellis.rdd import rasterize
from geopyspark.geotrellis import RasterizerOptions
ro = RasterizerOptions(includePartial=True, sampleType='PixelIsArea')
road_raster = rasterize(gps, geoms=libya_roads.geoms, 
                        crs="EPSG:3857", zoom=8, 
                        fill_value=1, cell_type="float32", 
                        options = ro)

road_raster.layer_metadata.bounds
# TODO: numPartitions is to wee small here

In [ ]:
road_raster = road_raster.repartition(num_partitions=20)

In [ ]:
road_pp = Pyramid(road_raster.pyramid(start_zoom=8, end_zoom=1))
road_png_pp = road_pp.to_png_pyramid(color_ramp='Viridis')
#road_png_pp.persist(StorageLevel.MEMORY_AND_DISK)
M.add_layer(RddRasterData(road_png_pp, name='Roads'))

In [ ]:
road_png_pp.unpersist()
M.remove_layer(M.layers[0])

In [ ]:
road_friction = (
    road_raster
    .reclassify({1:1}, data_type=int, replace_nodata_with=10)
)

In [ ]:
from geopyspark.geotrellis.render import PngRDD
png = PngRDD.makePyramid(road_friction, 'viridis', end_zoom=1)
png.cache()
from geonotebook.wrappers import RddRasterData
M.add_layer(RddRasterData(png, name='SRTM'))

In [ ]:
png.unpersist()
M.remove_layer(M.layers[0])

In [ ]:
with open('population.geojson', encoding='utf-8') as data_file:
    population_centers = MultiPoint([shape(geom['geometry']) for geom in json.load(data_file)['features']])

with open('conflict.geojson', encoding='utf-8') as data_file:
    conflict_centers=MultiPoint([shape(feature['geometry']) for feature in json.load(data_file)['features'] if feature['geometry'] != None])
population_centers
conflict_centers

In [ ]:
pop_cd = (
    road_friction
    .cost_distance(
        geometries = population_centers, 
        max_distance=1400000.0
    )
)

In [ ]:
con_cd = (
    road_friction
    .cost_distance(
        geometries = conflict_centers, 
        max_distance=1400000.0
    )
)

In [ ]:
from geopyspark.geotrellis.pyramid import Pyramid
from geopyspark.geotrellis.constants import *
pop_pp = Pyramid(pop_cd.pyramid(start_zoom=8, end_zoom=1))
con_pp = Pyramid(con_cd.pyramid(start_zoom=8, end_zoom=1))

In [ ]:
wo = (con_pp * 0.2) + (pop_pp * 0)
png=wo.to_png_pyramid(color_ramp='Viridis')

In [ ]:
#wo.pyramid[8].lookup(136, 110)

In [ ]:
from geonotebook.wrappers import RddRasterData
M.add_layer(RddRasterData(png, name='WO'))

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
from geopyspark.geotrellis.render import PngRDD
png = PngRDD.makePyramid(pop_cd, 'viridis', end_zoom=1)
png.cache()
from geonotebook.wrappers import RddRasterData
M.add_layer(RddRasterData(png, name='SRTM'))